# DATA CLEANING

#### This notebook cleans the race results data scraped from the Leipzig Marathon 'Ergebnis-Datenbank' (Results Database).

#### Multiple CSV files are imported and cleaned individually, then combined into a single CSV, ready for analysis in Tableau.

#### Transformations Applied:

- Converted Nettozeit (net time) into total seconds, compatible with Tableau
- Renamed columns for consistency and clarity
- Dropped unnecessary columns where necessary

In [1]:
# Import Libraries

import pandas as pd
import glob
import os
import re

In [3]:
# Get all race results csv files.

csv_files = glob.glob('Leipzig/*.csv')

In [5]:
# Sort the list in chronological order.

csv_files_sorted = sorted(csv_files, key=lambda x: int(re.search(r'(\d{4})', x).group()))

#### Cleaning 'Datum' (Date) Column:

In [ ]:
# Start by checking for any duplicate or incorrect values.

for file in csv_files_sorted:
    df = pd.read_csv(file)
    unique_datums = df['Datum'].unique()

    print(unique_datums)

In [9]:
# Race results from 1999 are missing. Checked the website and confirmed that this data is missing, not a code problem.

In [11]:
# Cleaning incorrect values from 2002. 

df = pd.read_csv(csv_files_sorted[24])

unique_datums = df['Datum'].unique()

print(unique_datums)

['28.04.2020' '28.04.2002' nan]


In [13]:
df['Datum'] = '28.04.2002'

df.to_csv(csv_files_sorted[34], index=False)

In [15]:
# Drop the 'virtueller Lauf 2020' and '2021' data, as these years will be too different to compare meaningfully.

csv_files_sorted.remove('Leipzig/scraped_data_2019.csv')
csv_files_sorted.remove('Leipzig/scraped_data_2020.csv')

In [17]:
# Change 2024 so that date is formatted consistently.

df = pd.read_csv(csv_files_sorted[-1])

df['Datum'] = '21.04.2024'

df.to_csv(csv_files_sorted[-1], index=False)

#### Cleaning 'Wettbewerb' (Competition) Column:

In [19]:
# Here I want to make sure that all data is only related to the Marathon race.
# This should have been taken care of during web scraping, but want to double check.
# Trying to avoid data from other events, e.g. half marathon or 10km.

In [21]:
for file in csv_files_sorted:
    df = pd.read_csv(file)
    unique_events = df['Wettbewerb'].unique()
    
    if len(unique_events) > 1:
        print(f"{file} has {len(unique_events)} unique Wettbewerb values: {unique_events}")

In [23]:
# Results as expected, data is formatted correctly.

#### Cleaning 'Platz Gesamt' (Overall Placing) Column:

In [25]:
# Dropping this column, as I will be able to calculate this myself in Tableau based on the finishing times.
# This will be more reliable, avoiding any transcription errors.

for file in csv_files_sorted:
    df = pd.read_csv(file)
    df = df.drop(columns='PlatzGesamt')
    df.to_csv(file, index=False)

#### Cleaning Empty Column:

In [ ]:
for file in csv_files_sorted:
    df = pd.read_csv(file)

    print(df['Name'].unique())

In [33]:
# This column is empty, so will be dropped.

In [35]:
for file in csv_files_sorted:
    df = pd.read_csv(file)
    df.drop(columns='Name', inplace=True)
    df.to_csv(file, index=False)

#### Cleaning Bib Number Column:

In [ ]:
 for file in csv_files_sorted:
    df = pd.read_csv(file)

    print(df['Vorname'][0])

In [39]:
# This seems to correspond to runner's bib numbers.
# To confirm this, I searched for an image of the 2022 winner, Nic Ihlow.
# His bib number was 40001, which agreed with the data.
# This column can be dropped.

In [41]:
for file in csv_files_sorted:
    df = pd.read_csv(file)
    df.drop(columns='Vorname', inplace=True)
    df.to_csv(file, index=False)

#### Cleaning 'Name' (Last Name) Column:

In [ ]:
for file in csv_files_sorted:
    df = pd.read_csv(file)

    print(df['Nettozeit'][0])

In [46]:
# This column is incorrectly lablled 'Nettozeit' (Net Time), but actually refers to last name.
# So will rename this column.

for file in csv_files_sorted:
    df = pd.read_csv(file)

    df = df.rename(columns={'Nettozeit': 'Name'})

    df.to_csv(file, index=False)

In [54]:
# There is a NaN showing up, which should be cleaned.
# Appears on only the first row.

In [68]:
file_path = csv_files_sorted[-2]
df = pd.read_csv(file_path)
df = df.iloc[1:]
df.to_csv(file_path, index=False)

#### Cleaning 'Vorname' (First Name) Column:

In [ ]:
for file in csv_files_sorted:
    df = pd.read_csv(file)

    print(df['Ort/Verein'][0])

In [74]:
# This column is incorrectly lablled 'Nettozeit' (Net Time), but actually refers to last name.
# So will rename this column.

for file in csv_files_sorted:
    df = pd.read_csv(file)

    df = df.rename(columns={'Ort/Verein': 'Vorname'})

    df.to_csv(file, index=False)

#### Cleaning Empty Column:

In [ ]:
for file in csv_files_sorted:
    df = pd.read_csv(file)

    print(df['Nation'][0])

In [80]:
for file in csv_files_sorted:
    df = pd.read_csv(file)
    df.drop(columns='Nation', inplace=True)
    df.to_csv(file, index=False)

#### Cleaning Net Time Column:

In [ ]:
for file in csv_files_sorted:
    df = pd.read_csv(file)

    print(df['Jahrgang'][0])

In [85]:
# This column is incorrectly lablled 'Jahrgang' (Age Cohort), but actually refers to net time.
# So will rename this column.

for file in csv_files_sorted:
    df = pd.read_csv(file)

    df = df.rename(columns={'Jahrgang': 'Nettozeit'})

    df.to_csv(file, index=False)

In [87]:
# Will convert these times from hh:mm:ss format to seconds, as this makes it easier to work with in Tableau.
# First, will need to ensure that all of these times are formatted correctly.

In [103]:
# Here I am performing three transformations:
# Convert all semi-colon and comma symbols to colons.
# Extract hh:mm:ss time
# Strip leading zero from hours

for file in csv_files_sorted:
    df = pd.read_csv(file)

    df['Nettozeit'] = df['Nettozeit'].astype(str).str.replace(';,', ':', regex=True)

    df['Nettozeit'] = df['Nettozeit'].str.extract(r'(\d{1,2}:\d{2}:\d{2})')

    df['Nettozeit'] = df['Nettozeit'].str.replace(r'^0(?=\d:)', '', regex=True)

    df.to_csv(file, index=False)

In [ ]:
for file in csv_files_sorted:
    df = pd.read_csv(file)

    print(df['Nettozeit'][0])

In [ ]:
# There might still be NaNs in the data, but I will treat these as DNFs (did not finish).

In [109]:
# Now I will convert this hh:mm:ss format to seconds.

for file in csv_files_sorted:
    df = pd.read_csv(file)

    df['Nettozeit'] = pd.to_timedelta(df['Nettozeit'], errors='coerce').dt.total_seconds()

    df.to_csv(file, index=False)

In [ ]:
for file in csv_files_sorted:
    df = pd.read_csv(file)

    print(df['Nettozeit'][0])

In [ ]:
# Want to manually check the conversion worked correctly for 2:37:07:

In [113]:
2 * 3600 + 37 * 60 + 7

9427

#### Cleaning Club Columns:

In [ ]:
for file in csv_files_sorted:
    df = pd.read_csv(file)

    print(df['Alterklasse'][0])

In [120]:
# This column is misnamed, actually refers to running club.
# Decided to drop this.

for file in csv_files_sorted:
    df = pd.read_csv(file)
    df.drop(columns='Alterklasse', inplace=True)
    df.to_csv(file, index=False)

#### Cleaning Age Cohort Column:

In [ ]:
for file in csv_files_sorted:
    df = pd.read_csv(file)

    print(df['PlatzAk'][0])

In [135]:
# This column is misnamed, actually refers to age cohort.

for file in csv_files_sorted:
    df = pd.read_csv(file)

    df = df.rename(columns={'PlatzAk': 'Alterklasse'})

    df.to_csv(file, index=False)

In [ ]:
for file in csv_files_sorted:
    df = pd.read_csv(file)

    print(df['Alterklasse'][0])

In [142]:
# Want to coerce invalid entries (like nationality) into NaNs.

for file in csv_files_sorted:
    df = pd.read_csv(file)
    df['Alterklasse'] = pd.to_numeric(df['Alterklasse'], errors='coerce')
    
    df.to_csv(file, index=False)

#### Drop Age Groups Column:

In [ ]:
for file in csv_files_sorted:
    df = pd.read_csv(file)

    print(df['Bruttozeit'].unique())

In [ ]:
# Here we have different age classes that were used within the first 10 years of the race.
# But these are very inconsistent and it's hard to find the exact definitions online.
# Also they are much less granular than age year, so I will just drop these.

In [148]:
for file in csv_files_sorted:
    df = pd.read_csv(file)
    df.drop(columns='Bruttozeit', inplace=True)
    df.to_csv(file, index=False)

#### Merge all CSV files together:

In [ ]:
# The final step in my data cleaning is to merge all the csvs together to make one list which can then be imported into Tableau.

In [156]:
dfs = []

for file in csv_files_sorted:
    df = pd.read_csv(file)
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)

combined_df.to_csv("final.csv", index=False)

#### 